A catalog named telecom_catalog_assign
A schema landing_zone
A volume landing_vol
Using dbutils.fs.mkdirs, create folders:
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/ /Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/ /Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/
Explain the difference between (Just google and understand why we are going for volume concept for prod ready systems):
a. Volume vs DBFS/FileStore
## # b. Why production teams prefer Volumes for regulated data

In [0]:
spark.sql("CREATE CATALOG IF NOT EXISTS telecom_catalog_assign")
spark.sql("CREATE SCHEMA IF NOT EXISTS  telecom_catalog_assign.landing_zone")
spark.sql(
    "CREATE VOLUME if not exists telecom_catalog_assign.landing_zone.landing_vol"
)

# Now creat
# Then create the directory
#dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/")
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/")
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/")



In [0]:
customer_csv = '''
101,Arun,31,Chennai,PREPAID
102,Meera,45,Bangalore,POSTPAID
103,Irfan,29,Hyderabad,PREPAID
104,Raj,52,Mumbai,POSTPAID
105,,27,Delhi,PREPAID
106,Sneha,abc,Pune,PREPAID
'''

# Define the DBFS path for the new file
dbfs_path = "dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customers_2.csv" # A common location for user uploads

# Use dbutils.fs.put() to write the string content to DBFS
# The 'True' argument means 'overwrite' (optional, set to True to replace existing file)
dbutils.fs.put(dbfs_path, customer_csv.strip(), True)

print(f"Data successfully written to {dbfs_path}")

In [0]:
usage_tsv ='''
customer_id\tvoice_mins\tdata_mb\tsms_count
101\t320\t1500\t20
102\t120\t4000\t5
103\t540\t600\t52
104\t45\t200\t2
105\t0\t0\t0
'''
dbfs_path = "dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.tsv"
dbutils.fs.put(dbfs_path, usage_tsv.strip(), True)

print(f"Data successfully written to {dbfs_path}")

tower_logs_region1 = '''
event_id|customer_id|tower_id|signal_strength|timestamp
5001|101|TWR01|-80|2025-01-10 10:21:54
5005|101|TWR01|-80|2025-12-10 10:41:54
'''
dbfs_path = "dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region1.csv"
dbutils.fs.put(dbfs_path, tower_logs_region1.strip(), True)

print(f"Data successfully written to {dbfs_path}")



In [0]:
dbutils.fs.ls("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer")
dbutils.fs.ls("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage")
dbutils.fs.ls("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower")



In [0]:
tower_logs_region2 = '''
event_id|customer_id|tower_id|signal_strength|timestamp
5004|104|TWR05|-75|2025-01-10 11:01:12
'''
dbfs_path = "dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region2.csv"
dbutils.fs.put(dbfs_path, tower_logs_region2.strip(), True)

print(f"Data successfully written to {dbfs_path}")

Read all tower logs using: Path glob filter (example: *.csv) Multiple paths input Recursive lookup

Demonstrate these 3 reads separately: Using pathGlobFilter Using list of paths in spark.read.csv([path1, path2]) Using .option("recursiveFileLookup","true")

Compare the outputs and understand when each should be used.

In [0]:

df_multiple_sources=spark.read.csv(path=["/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region1.csv","dbfs:////Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region2.csv"],inferSchema=True,header=True,sep='|',pathGlobFilter="*.csv",recursiveFileLookup=True)
#.toDF("cid","fn","ln","a","p")
print(df_multiple_sources.count())
df_multiple_sources.show(2)

In [0]:
'''Demonstrate these 3 reads separately: Using pathGlobFilter Using list of paths in spark.read.csv([path1, path2]) Using .option("recursiveFileLookup","true")'''
df_multiple_sources=spark.read.csv(path=["/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region1.csv","dbfs:////Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region2.csv"],inferSchema=True,header=True,sep='|',recursiveFileLookup=True)
print(df_multiple_sources.count())
df_multiple_sources.show(2)

In [0]:

csv_df1=spark.read.csv(path="dbfs:////Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region*.csv",inferSchema=True,header=True,sep='|')
print(csv_df1.count())
csv_df1.show()

In [0]:
csv_df1=spark.read.csv(path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customers.csv",inferSchema=False,header=False,sep=',').toDF("id","name","age","loc","plan")
df2=csv_df1.where("age!='abc' ")
csv_df1.printSchema()
print(csv_df1.count())
csv_df1.show()
df2.show()

In [0]:
csv_df1=spark.read.csv(path="dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.tsv",inferSchema=True,header=True,sep='\t')
print(csv_df1.count())
csv_df1.show()

In [0]:
df_recursive = spark.read.format("csv") .option("header", "true") .option("delimiter", "|") .option("recursiveFileLookup", "true") .load("dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/")

print("--- DataFrame using recursiveFileLookup='true' ---")
df_recursive.show()

In [0]:
df_glob = spark.read.format("csv") .option("header", "true")  .option("delimiter", "|") .option("pathGlobFilter", "*region*.csv") .load("dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/")

print("--- DataFrame using pathGlobFilter ('region*.csv') ---")
df_glob.show()

Apply column names using string using toDF function for customer data
Apply column names and datatype using the schema function for usage data
Apply column names and datatype using the StructType with IntegerType, StringType, TimestampType and other classes for towers data

In [0]:

#customer_id|voice_mins|data_mb|sms_count
str_struct="customer_id integer,voice_mins integer,data_mb integer,sms_count integer"
csv_df1=spark.read.schema(str_struct).csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.tsv",header=True,sep='\t')
print(csv_df1.printSchema())
csv_df1.show()


#2. Important part - Using structure type to define custom complex schema.
#4000001,Kristina,Chung,55,Pilot
#pattern - 
#import the types library based classes..
# define_structure=StructType([StructField("colname",DataType(),True),StructField("colname",DataType(),True)...])



In [0]:
#event_id|customer_id|tower_id|signal_strength|          timestamp
from pyspark.sql.types import StructType,StructField,StringType,IntegerType
custom_schema=StructType([StructField("event_id",IntegerType(),False),StructField("customer_id",IntegerType(),False),StructField("tower_id",StringType(),False),StructField("signal_strength",IntegerType(),False),StructField("timestamp",StringType(),False)])
csv_df1=spark.read.schema(custom_schema).csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/",pathGlobFilter="*.csv",header='True',sep='|')
print(csv_df1.printSchema())
csv_df1.show()

6. Write Operations (Data Conversion/Schema migration) – CSV Format Usecases
Write customer data into CSV format using overwrite mode
Write usage data into CSV format using append mode
Write tower data into CSV format with header enabled and custom separator (|)
Read the tower data in a dataframe and show only 5 rows.
Download the file into local from the catalog volume location and see the data of any of the above files opening in a notepad++.

In [0]:
from pyspark.sql.types import StructType, StructField, StringType

schema = StructType([
    StructField("id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("age", StringType(), True),
    StructField("loc", StringType(), True),
    StructField("plan", StringType(), True)
])

csv_df1 = spark.read.csv(
    path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customers.csv",
    schema=schema,
    header=False,
    sep=','
)

df2 = csv_df1.where("age != 'abc'")

display(csv_df1)
display(df2)

In [0]:
csv_df1 = spark.read.csv(
    path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customers_2.csv",
    inferSchema=True, 
    header=False, 
    sep=','
)

In [0]:

# 3. Now you can safely overwrite
csv_df1.write.csv(
    path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customercsv",
    sep=',',
    header=False,
    mode='overwrite',
    compression='gzip')

In [0]:
csv_df1=spark.read.csv(path="dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.tsv",inferSchema=True,header=True,sep='\t')

csv_df1.write.csv(
    path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usagetsv",
    sep=',',
    header=True,
    mode='append',
    compression='gzip')

Write tower data into CSV format with header enabled and custom separator (|)
Read the tower data in a dataframe and show only 5 rows.
Download the file into local from the catalog volume location and see the data of any of the above files opening in a notepad++.

In [0]:
#event_id|customer_id|tower_id|signal_strength|          timestamp
from pyspark.sql.types import StructType,StructField,StringType,IntegerType
custom_schema=StructType([StructField("event_id",IntegerType(),False),StructField("customer_id",IntegerType(),False),StructField("tower_id",StringType(),False),StructField("signal_strength",IntegerType(),False),StructField("timestamp",StringType(),False)])
csv_df1=spark.read.schema(custom_schema).csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/",pathGlobFilter="*.csv",header='True',sep='|')
print(csv_df1.printSchema())
csv_df1.show()

csv_df1.write.csv(
    path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/towercsv",
    sep='|',
    header=True,
    mode='append',
    compression='gzip')

display(csv_df1.show(5))


7. Write Operations (Data Conversion/Schema migration)– JSON Format Usecases
Write customer data into JSON format using overwrite mode
Write usage data into JSON format using append mode and snappy compression format
Write tower data into JSON format using ignore mode and observe the behavior of this mode
Read the tower data in a dataframe and show only 5 rows.
Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.


In [0]:
csv_df1 = spark.read.csv(
    path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customers_2.csv",
    inferSchema=True, 
    header=False, 
    sep=','
)

csv_df1.write.json(
    path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customersjson",
    mode='overwrite',
    compression='gzip')

csv_df1.write.parquet(
    path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customersparquet",
    mode='overwrite',
    compression='gzip')

csv_df1.write.orc(
    path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customersorc",
    mode='overwrite',
    compression='zlib')
    
csv_df1.write .format("delta") .mode("overwrite") .save("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customersdelta")

In [0]:
csv_df1=spark.read.csv(path="dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.tsv",inferSchema=True,header=True,sep='\t')
csv_df1.write.json(path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usagetsvjson",mode='append',compression='snappy')
csv_df1.write.parquet(path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usagetsvparquet",mode='append',compression='snappy')
csv_df1.write.orc(path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usagetsvorc",mode='append',compression='snappy')

csv_df1.write.format("delta") .mode("append").save("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usagetsdelta")


In [0]:
from pyspark.sql.types import StructType,StructField,StringType,IntegerType
custom_schema=StructType([StructField("event_id",IntegerType(),False),StructField("customer_id",IntegerType(),False),StructField("tower_id",StringType(),False),StructField("signal_strength",IntegerType(),False),StructField("timestamp",StringType(),False)])
csv_df1=spark.read.schema(custom_schema).csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/",pathGlobFilter="*.csv",header='True',sep='|')
print(csv_df1.printSchema())
csv_df1.show()
csv_df1.write.json(path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/towercsvjson",mode='ignore',compression='snappy')
csv_df1.write.parquet(path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/towercsvparquet",mode='ignore',compression='snappy')
display(csv_df1)
csv_df1.write.orc(path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/towercsvorc",mode='ignore',compression='snappy')

csv_df1.write.format("delta") .mode("ignore").save("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/towercsvdelta")

Write customer data into Parquet format using overwrite mode and in a gzip format
Write usage data into Parquet format using error mode
Write tower data into Parquet format with gzip compression option
Read the usage data in a dataframe and show only 5 rows.
Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.



In [0]:

csv_df1 = spark.read.csv(
    path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customers_2.csv",
    inferSchema=True, 
    header=False, 
    sep=','
)
csv_df1.write.parquet(
    path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customersparquet",
    mode='overwrite',
    compression='gzip')


    

In [0]:
csv_df1=spark.read.csv(path="dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.tsv",inferSchema=True,header=True,sep='\t')

csv_df1.write.parquet(path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usagetsvparquet_cp",mode='error',compression='snappy')

In [0]:
from pyspark.sql.types import StructType,StructField,StringType,IntegerType
custom_schema=StructType([StructField("event_id",IntegerType(),False),StructField("customer_id",IntegerType(),False),StructField("tower_id",StringType(),False),StructField("signal_strength",IntegerType(),False),StructField("timestamp",StringType(),False)])
csv_df1=spark.read.schema(custom_schema).csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/",pathGlobFilter="*.csv",header='True',sep='|')
print(csv_df1.printSchema())
csv_df1.show()
csv_df1.write.parquet(path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/towercsvparquet",mode='ignore',compression='gzip')
display(csv_df1)

In [0]:
par_df1=spark.read.parquet("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usagetsvparquet_cp")
par_df1.show(5)
display(par_df1.take(5))





Write customer data using saveAsTable() as a managed table
Write usage data using saveAsTable() with overwrite mode
Drop the managed table and verify data removal
Go and check the table overview and realize it is in delta format in the Catalog.
Use spark.read.sql to write some simple queries on the above tables created.

In [0]:
csv_df1 = spark.read.csv(
    path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customers_2.csv",
    inferSchema=True, 
    header=False, 
    sep=','
)
csv_df1.write.saveAsTable("workspace.wd36schema2.lh_custtbl1",mode='overwrite')

spark.sql("select * from workspace.wd36schema2.lh_custtbl1").show()
display(spark.sql("show create table workspace.wd36schema2.lh_custtbl1"))
#csv_df1.write.droptable("workspace.wd36schema2.lh_custtbl1",True)
#spark.sql("drop table if exists workspace.wd36schema2.lh_custtbl1")


In [0]:
csv_df1=spark.read.csv(path="dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.tsv",inferSchema=True,header=True,sep='\t')

csv_df1.write.saveAsTable("workspace.wd36schema2.lh_usagetbl1",mode='overwrite')
spark.sql("select * from workspace.wd36schema2.lh_usagetbl1").show()
spark.sql("create table if not exists workspace.wd36schema2.lh_usagetbl2 as select * from workspace.wd36schema2.lh_usagetbl1 where 1=0")
spark.sql("select * from workspace.wd36schema2.lh_usagetbl2").show()
display(spark.sql("show create table workspace.wd36schema2.lh_usagetbl2"))
csv_df1.write.insertInto("workspace.wd36schema2.lh_usagetbl2",overwrite=True)
spark.sql("select * from workspace.wd36schema2.lh_usagetbl2").show()
display(spark.sql("show create table workspace.wd36schema2.lh_usagetbl2"))


Write customer data using insertInto() in a new table and find the behavior
Write usage data using insertTable() with overwrite mode

In [0]:
csv_df1 = spark.read.csv(
    path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customers_2.csv",
    inferSchema=True, 
    header=False, 
    sep=','
)
csv_df1.write.saveAsTable("workspace.wd36schema2.lh_custtbl1",mode='overwrite')

spark.sql("select * from workspace.wd36schema2.lh_custtbl1").show()
display(spark.sql("show create table workspace.wd36schema2.lh_custtbl1"))
spark.sql("""
    CREATE TABLE IF NOT EXISTS workspace.wd36schema2.lh_custtbl2
    AS SELECT * FROM workspace.wd36schema2.lh_custtbl1 WHERE 1=0
""")
display(spark.sql("select * from workspace.wd36schema2.lh_custtbl2" ))
csv_df1.write.insertInto("workspace.wd36schema2.lh_custtbl2",overwrite=True)
display(spark.sql("select * from workspace.wd36schema2.lh_custtbl2" ))

In [0]:
Write customer data into XML format using rowTag as cust
Write usage data into XML format using overwrite mode with the rowTag as usage
Download the xml data and open the file in notepad++ and see how the xml file looks like.

In [0]:
csv_df1 = spark.read.csv(
    path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customers_2.csv",
    inferSchema=True, 
    header=False, 
    sep=','
)
csv_df1.write.xml("/Volumes/workspace/wd36schema2/ingestion_volume/target/xmlout",mode="ignore",rowTag="cust")
csv_df2=spark.read.format("xml").option("rowTag","cust").load("/Volumes/workspace/wd36schema2/ingestion_volume/target/xmlout")
csv_df2.show()
     

In [0]:
csv_df1=spark.read.csv(path="dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.tsv",inferSchema=True,header=True,sep='\t')
csv_df1.write.xml("/Volumes/workspace/wd36schema2/ingestion_volume/target/usagexmlout",mode="overwrite",rowTag="usage")
csv_df2=spark.read.format("xml").option("rowTag","usage").load("/Volumes/workspace/wd36schema2/ingestion_volume/target/usagexmlout")
csv_df2.show()

In [0]:
read_orc_cust_df=spark.read.orc("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customersorc")
read_orc_cust_df.take(2)
write_cust_orc_pqt=read_orc_cust_df.write.parquet("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customersorcparquet",mode="overwrite")
csv_df2=spark.read.format("parquet").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customersorcparquet")
#display(csv_df2.show(5))

#csv_df2.collect()
# Removed .show() on write_cust_orc_pqt, as write.parquet() returns None
